In [23]:
import pandas as pd
from sklearn.cluster import KMeans
import json

In [24]:
file_path = 'bandgap_y_sparse_single_50_clusters.json'
n_clusters = 50

# Read the data from the CSV file
df_data = pd.read_csv("bandgap_features.csv")

excluded = ["formula", "composition", "formula", "gap expt"]
y = df_data['gap expt'].values
X = df_data.drop(excluded, axis=1)

In [25]:
y.shape

(4604,)

In [26]:
import numpy as np
from scipy.stats import gaussian_kde

# Generate a sample one-dimensional vector
#np.random.seed(42)
#vector = np.random.randn(100)

# Estimate the density using kernel density estimation
kde = gaussian_kde(y)

# Evaluate the estimated density at sample points
sample_points = np.linspace(y.min(), y.max(), 100)
densities = kde(sample_points)

# Print the estimated density values
# for i in range(len(sample_points)):
#     print(f"Density at sample point {sample_points[i]}: {density[i]}")


In [27]:


density_dict = {}

# Print the density values for each sample position
for i in range(len(densities)):
    density = densities[i]
    #print(f"Density at sample position {i+1}: {density}")
    
    density_dict[i] = density

#print(density_dict)

sorted_density = sorted(density_dict.items(), key=lambda x: x[1])
#print(sorted_density)
top500 = [x[0] for x in sorted_density[:500]]
X_sparse = X.iloc[top500]
print(X_sparse.shape)

(100, 133)


In [28]:
id_map = {}

for i, index in enumerate(top500):
    id_map[i] = top500[i]
    
#id_map

In [29]:
# Perform KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, max_iter=500, random_state=222)
kmeans.fit(X_sparse)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# Print cluster labels and centers
#print("Cluster Labels:", labels)
#print("Cluster Centers:", centers)

# Create a dictionary to store the cluster assignments
cluster = {}
#print(labels)
for i, cluster_id in enumerate(labels):
    if cluster_id in cluster:
        #cluster[int(cluster_id)].append(i)
        continue
    else:
        cluster[int(cluster_id)] = [id_map[i]]

# Print the size of each cluster
#for key in sorted(cluster):
    #print(key, (cluster[key]))

# Example: Print formulas in cluster 0
#for index in cluster[0]:
    #print(df_data.iloc[index]["formula"])

# Save the dictionary to a file in JSON format
with open(file_path, 'w') as file:
    json.dump(cluster, file)